In [1]:
from casadi import *
import os
from dynamic_model import *
import Bezier
from readDataFcn import getTrack
%matplotlib qt

In [2]:
%matplotlib qt
table = Bezier.generatelookuptable(1)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +5e+03  2e-01  1e-06  1e+00  2e+03    ---    ---    3  1  - |  -  - 
 1  -2.346e-01  +9.853e-03  +5e+01  2e-04  1e-08  3e-01  3e+01  0.9890  1e-04   3  4  4 |  0  0
 2  +5.195e-01  +5.432e-01  +4e+00  1e-06  9e-09  2e-02  2e+00  0.9330  3e-04   6  6  6 |  0  0
 3  +3.236e+00  +3.290e+00  +4e-01  2e-07  1e-09  5e-02  2e-01  0.9890  2e-02   1  1  1 |  0  0
 4  +3.894e+00  +3.895e+00  +5e-03  7e-08  9e-10  1e-03  3e-03  0.9857  1e-04   3  4  4 |  0  0
 5  +3.906e+00  +3.906e+00  +1e-04  2e-09  9e-11  2e-05  7e-05  0.9808  2e-04   2  2  2 |  0  0
 6  +3.906e+00  +3.906e+00  +8e-06  2e-10  7e-11  2e-06  4e-06  0.9393  4e-03   2  2  2 |  0  0
 7  +3.906e+00  +3.906e+00  +1e-07  5e-11  2e-11  8e-08  8e-08  0.9887  8e-03   2  2  2 |  0  0
 8  +3.906e+00  +3.906e+00  +3e-09  2e-11  3e-

In [3]:
table

array([[ 0.00000000e+00,  0.00000000e+00,  5.00000000e-02, ...,
         9.50911506e-01, -2.20724286e-02, -4.20724286e-02],
       [ 1.00534878e-02, -3.68123441e-02,  5.00000000e-02, ...,
         9.43578656e-01, -2.06215084e-02, -4.06215084e-02],
       [ 2.01069756e-02, -6.26666828e-02,  5.00000000e-02, ...,
         9.37290942e-01, -1.94371485e-02, -3.94371485e-02],
       ...,
       [ 3.21711610e+00,  2.58622825e+01,  4.25359341e-02, ...,
         8.26299228e-01, -3.99571036e-03, -2.39957104e-02],
       [ 3.22716959e+00,  2.59327249e+01,  4.64249195e-02, ...,
         8.98522534e-01, -1.29806819e-02, -3.29806819e-02],
       [ 3.23722307e+00,  2.60000000e+01,  4.99948064e-02, ...,
         0.00000000e+00,  5.99872614e-02,  3.99872614e-02]])

In [4]:
track = 0
N = 10
Tf = 0.5

#load track
waypoints = Bezier.getwaypoints(track)
#abez,bbez coeffs
a_bez, b_bez = Bezier.interpolate(waypoints)
order_inverse = 4
norm_inverse = 2
coeffs = Bezier.fit_st(order_inverse, norm_inverse, waypoints)

# define casadi struct
model = types.SimpleNamespace()
constraints = types.SimpleNamespace()

model_name = "f110_dynamic_model"
#loadparameters
m = 2 #[kg]
lf = 0.1 #[m]
lr = 0.1 #[m]
Iz = 1 #[kg*m^3]

#pajecka and motor coefficients
Bf = 1
Br = 1
Cf = 1
Cr = 1
Cm1 = 1
Cm2 = 2
Cr = 1
Cd = 1
Df = 1
Dr = 1

TypeError: fit_st() missing 2 required positional arguments: 'a' and 'b'

In [7]:
# define casadi struct
model = types.SimpleNamespace()
constraints = types.SimpleNamespace()

model_name = "f110_dynamic_model"
#loadparameters
m = 2 #[kg]
lf = 0.1 #[m]
lr = 0.1 #[m]
Iz = 1 #[kg*m^3]

#pajecka and motor coefficients
Bf = 1
Br = 1
Cf = 1
Cr = 1
Cm1 = 1
Cm2 = 2
Cr = 1
Cd = 1
Df = 1
Dr = 1


#parameter vector
xt =  SX.sym("xt")
yt =  SX.sym("yt")
phit = SX.sym("phit")
sin_phit = SX.sym("sin_phit")
cos_phit = SX.sym("cos_phit")
gt_upper = SX.sym("gt_upper")
gt_lower = SX.sym("gt_lower")
#stores linearization point
theta_hat = SX.sym("theta_hat")
Qc = SX.sym("Qc")
Ql = SX.sym("Ql")
R = SX.sym("R")

#single track model with pajecka tireforces as in  Optimization-Based Autonomous Racing of 1:43 Scale RC Cars Alexander Liniger, Alexander Domahidi and Manfred Morari
#pose
posx = SX.sym("posx")
posy = SX.sym("posy")


#vel (long and lateral)
vx = SX.sym("vx")
vy = SX.sym("vy")

#body angular Rate
omega = SX.sym("omega")
#heading
phi = SX.sym("phi")

#virtual input projected track velocity
vt = SX.sym("vt")

#steering_angle
delta = SX.sym("delta")

#motorinput
d = SX.sym("d")

#dynamic forces
Frx = SX.sym("Frx")
Fry = SX.sym("Fry")
Ffx = SX.sym("Ffx")
Ffy = SX.sym("Ffy")

#arclength progress
theta = SX.sym("theta")

#temporal derivatives
posxdot = SX.sym("xdot")
posydot = SX.sym("ydot")
vxdot = SX.sym("vxdot")
vydot = SX.sym("vydot")
phidot = SX.sym("phidot")
omegadot = SX.sym("omegadot")
thetadot = SX.sym("thetadot")
deltadot = SX.sym("deltadot")

#car state Dynamics
x = vertcat(
    posx,
    posy,
    phi,
    vx,
    vy,
    omega,
    theta
    )

xdot = vertcat(
    posxdot,
    posydot,
    phidot,
    vxdot,
    vydot,
    omegadot,
    thetadot
    )



#front lateral tireforce
alphaf = -atan((omega*lf + vy)/vx) + delta
Ffy = Df*sin(Cf*atan(Bf*alphaf))

#rear lateral tireforce
alphar = atan((omega*lr - vy)/vx)
Fry = Dr*sin(Cf*atan(Br*alphar))

#rear longitudinal forces
Frx = (Cm1-Cm2*vx) * d - Cr -Cd*vx**2

f_expl = vertcat(
    vx*cos(phi) - vy * sin(phi),
    vx*sin(phi) + vy * cos(phi),
    omega,
    1/m * (Frx - Fry*sin(delta) + m*vy*omega),
    1/m * (Fry + Fry*cos(delta) - m*vx*omega),
    1/Iz * (Ffy*lf*cos(delta) - Fry*lr),
    vt
    )


model.f_expl_expr = f_expl
model.f_impl_expr = xdot - f_expl
model.x = x

#halfspace constraints on x capturing the track
n = vertcat(-sin_phit, cos_phit)
constraints.C = np.matrix([[n[0], n[1], 0, 0, 0, 0, 0],  [-n[0], -n[1], 0, 0, 0, 0, 0]])
constraints.ge_upper = vertcat(gt_upper, -gt_lower)


#compute approximate linearized contouring and lag error
xt_hat = xt + cos_phit * ( theta - theta_hat)
yt_hat = yt + sin_phit * ( theta - theta_hat)

e_cont = sin_phit * (posx - xt_hat) - cos_phit * (posy - yt_hat)
e_lag = -cos_phit * (posx - xt_hat) - sin_phit * (posy - yt_hat)

error = vertcat(e_cont, e_lag)
#set up stage cost
Q = diag(vertcat(Qc, Ql))
model.stage_cost = bilin(Q, error, error)


In [12]:
Q

SX(
[[Qc, 00], 
 [00, Ql]])

In [43]:
tvals = np.linspace(0, 1, 26)
kapparef_s = interpolant("kapparef_s","bspline", [tvals], waypoints[:,0].tolist())


RuntimeError: .../casadi/core/function_internal.cpp:70: Function name is not valid. A valid function name is a string starting with a letter followed by letters, numbers or non-consecutive underscores. It may also not match the keywords 'null', 'jac' or 'hess'. Got ''

In [23]:
xdot

MX(vertcat(posxdot, posydot, phidot, vxdot, vydot, omegadot))

In [48]:
# load track parameters
track="LMS_Track.txt"
[s0, _, _, _, kapparef] = getTrack(track)
length = len(s0)
pathlength = s0[-1]



In [51]:
# load track parameters
[s0, _, _, _, kapparef] = getTrack(track)
length = len(s0)
pathlength = s0[-1]
# copy loop to beginning and end
s0 = np.append(s0, [s0[length - 1] + s0[1:length]])
kapparef = np.append(kapparef, kapparef[1:length])
s0 = np.append([-s0[length - 2] + s0[length - 81 : length - 2]], s0)
kapparef = np.append(kapparef[length - 80 : length - 1], kapparef)

# compute spline interpolations
kapparef_s = interpolant("kapparef_s", "bspline", [s0], kapparef)

RuntimeError: .../casadi/core/function_internal.cpp:70: Function name is not valid. A valid function name is a string starting with a letter followed by letters, numbers or non-consecutive underscores. It may also not match the keywords 'null', 'jac' or 'hess'. Got ''

In [53]:
s0

array([-3.1307295, -3.1061956, -3.0436956, -2.9811956, -2.9186956,
       -2.8561956, -2.7936956, -2.7311956, -2.6686956, -2.6061956,
       -2.5816618, -2.557128 , -2.5325941, -2.5080603, -2.4835265,
       -2.4589926, -2.4344588, -2.409925 , -2.3853911, -2.3608573,
       -2.3363234, -2.3117896, -2.2872558, -2.2627219, -2.2381881,
       -2.2136543, -2.1891204, -2.1645866, -2.1400527, -2.1155189,
       -2.0909851, -2.0664512, -2.0419174, -2.0173836, -1.9928497,
       -1.9683159, -1.943782 , -1.9192482, -1.8947144, -1.8701805,
       -1.8456467, -1.8211129, -1.7586129, -1.6961129, -1.6336129,
       -1.5711129, -1.5086129, -1.4461129, -1.3836129, -1.3211129,
       -1.2586129, -1.1961129, -1.1336129, -1.0711129, -1.0086129,
       -0.9461129, -0.8836129, -0.8211129, -0.796579 , -0.7720452,
       -0.7475113, -0.7229775, -0.6984437, -0.6739098, -0.649376 ,
       -0.6248422, -0.6003083, -0.5757745, -0.5512407, -0.5267068,
       -0.502173 , -0.4776391, -0.4531053, -0.4285715, -0.3571